# AdaptivePrecision 4D Development & Testing Notebook

**Interactive development environment for testing AdaptivePrecision with 4D Shubert function**

This notebook provides:
- **Revise.jl integration** for automatic code reloading
- **Runtime breakdown analysis** with detailed profiling
- **Accuracy comparison** between Float64Precision and AdaptivePrecision
- **Interactive parameter tuning** for optimal performance
- **Visual analysis** of results and performance metrics

## 🔧 Development Workflow:
1. **Setup** - Load packages with Revise
2. **Baseline** - Test current implementation
3. **Iterate** - Modify code, test immediately
4. **Profile** - Analyze performance bottlenecks
5. **Compare** - Validate accuracy improvements

## 📦 Setup: Packages and Revise Integration

In [3]:
include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))

  Activating project at `~/globtim`


✅ BenchmarkTools available - full benchmarking features enabled

🎯 4D AdaptivePrecision Testing Framework Loaded!
📋 Type show_4d_framework_functions() to see all available functions
🚀 Quick start: run_4d_quick_test()
💡 Short aliases available: help_4d(), quick_test(), compare_precisions(), scaling_analysis(), sparsity_analysis()
✅ Development environment ready with Revise integration


## 🎯 Phase 2A: Shubert 4D Baseline Testing

In [4]:
# Test configuration
const n = 4  # 4D problem
const scale_factor = 2.0
const center = zeros(4)

# Testing parameters - conservative for development
degrees_to_test = [3, 4]
samples_to_test = [10, 12, 14]

@printf "Configuration: %dD, scale=%.1f, degrees=%s, samples=%s\n" n scale_factor degrees_to_test samples_to_test

Configuration: 4D, scale=2.0, degrees=[3, 4], samples=[10, 12, 14]


### 🔍 Single Test Case - Detailed Analysis

In [5]:
# Single test case for detailed analysis
test_degree = 3  # Increased from 3 for better testing
test_samples = 20

println("🔬 Testing: Shubert 4D (degree=$test_degree, samples=$test_samples)")

# Create test input
@time TR = test_input(
    shubert_4d,
    dim = n,
    center = center,
    GN = test_samples,
    sample_range = scale_factor,
    degree_max = test_degree + 2
)

@printf "✅ Test input: %d samples, range=%.1f\n" TR.GN scale_factor

🔬 Testing: Shubert 4D (degree=3, samples=20)
  0.036604 seconds (155.71 k allocations: 8.547 MiB, 98.16% compilation time)
✅ Test input: 20 samples, range=2.0


### ⏱️ Runtime Breakdown: Construction Phase

In [ ]:
# Construction phase with timeout protection
println("\n⏱️ Construction Phase Analysis")

# Timeout protection function
function safe_constructor(TR, degree, precision; timeout_seconds=60)
    task = @async Constructor(TR, degree, basis=:chebyshev, precision=precision, verbose=0)
    start_time = time()
    
    while !istaskdone(task)
        if time() - start_time > timeout_seconds
            @warn "Constructor timed out after $timeout_seconds seconds"
            return nothing
        end
        sleep(0.1)
    end
    
    return fetch(task)
end

# Test Float64Precision first
println("🔍 Float64Precision:")
GC.gc()
@time pol_float64 = safe_constructor(TR, test_degree, Float64Precision)

if pol_float64 !== nothing
    @printf "  ✅ Success: L2=%.2e, coeffs=%d\n" pol_float64.nrm length(pol_float64.coeffs)
else
    println("  ❌ Failed or timed out")
    error("Float64Precision construction failed - check parameters")
end


⏱️ Construction Phase Analysis
🔍 Float64Precision:


In [ ]:
# Detailed benchmarking and comparison
println("\n📊 Detailed Performance Analysis")

# Benchmark Float64Precision
println("🔍 Benchmarking Float64Precision...")
GC.gc()
bench_f64 = @benchmark Constructor($TR, $test_degree, basis=:chebyshev, precision=Float64Precision, verbose=0) samples=3

# Test AdaptivePrecision with timeout
println("🚀 Testing AdaptivePrecision...")
GC.gc()
@time pol_adaptive = safe_constructor(TR, test_degree, AdaptivePrecision)

if pol_adaptive !== nothing
    # Benchmark AdaptivePrecision
    bench_adaptive = @benchmark Constructor($TR, $test_degree, basis=:chebyshev, precision=AdaptivePrecision, verbose=0) samples=3
    
    # Performance comparison
    time_overhead = median(bench_adaptive.times) / median(bench_f64.times)
    memory_overhead = bench_adaptive.memory / bench_f64.memory
    norm_diff = abs(pol_adaptive.nrm - pol_float64.nrm)
    
    @printf "\n📈 Performance Summary:\n"
    @printf "  Float64:    %.3fs, %d MB, L2=%.2e\n" median(bench_f64.times)/1e9 bench_f64.memory÷1024^2 pol_float64.nrm
    @printf "  Adaptive:   %.3fs, %d MB, L2=%.2e\n" median(bench_adaptive.times)/1e9 bench_adaptive.memory÷1024^2 pol_adaptive.nrm
    @printf "  Overhead:   %.2fx time, %.2fx memory\n" time_overhead memory_overhead
    @printf "  L2 diff:    %.2e\n" norm_diff
else
    println("❌ AdaptivePrecision construction failed")
end

### 🔄 Runtime Breakdown: Monomial Conversion Phase

In [ ]:
# Monomial conversion analysis (only if AdaptivePrecision succeeded)
if pol_adaptive !== nothing
    println("\n🔄 Monomial Conversion Analysis")
    
    @polyvar x[1:n]
    
    # Convert both polynomials to monomial basis
    println("Converting to monomial basis...")
    GC.gc()
    @time mono_float64 = to_exact_monomial_basis(pol_float64, variables=x)
    @time mono_adaptive = to_exact_monomial_basis(pol_adaptive, variables=x)
    
    # Extract coefficients
    coeffs_f64 = [coefficient(t) for t in terms(mono_float64)]
    coeffs_adaptive = [coefficient(t) for t in terms(mono_adaptive)]
    
    @printf "\n📊 Monomial Conversion Results:\n"
    @printf "  Float64:    %d terms, type=%s\n" length(coeffs_f64) typeof(coeffs_f64[1])
    @printf "  Adaptive:   %d terms, type=%s\n" length(coeffs_adaptive) typeof(coeffs_adaptive[1])
    @printf "  Match:      %s\n" length(coeffs_f64) == length(coeffs_adaptive)
    
    # Show sample coefficients
    if length(coeffs_adaptive) > 0
        println("\n🔍 Sample BigFloat coefficients:")
        for i in 1:min(3, length(coeffs_adaptive))
            @printf "  [%d]: %.6e\n" i Float64(coeffs_adaptive[i])
        end
    end
else
    println("\n⚠️ Skipping monomial conversion (AdaptivePrecision failed)")
end

### 📈 Coefficient Analysis and Sparsity

In [ ]:
# Coefficient analysis and sparsity (only if conversion succeeded)
if pol_adaptive !== nothing && @isdefined(coeffs_adaptive) && length(coeffs_adaptive) > 0
    println("\n📈 Coefficient Analysis")
    
    # Analyze coefficient distribution
    @time analysis = analyze_coefficient_distribution(mono_adaptive)
    
    @printf "\n📊 Statistics: %d terms, range %.2e to %.2e\n" analysis.n_total analysis.min_coefficient analysis.max_coefficient
    
    # Test sparsification at key thresholds
    println("\n✂️ Sparsity Analysis:")
    thresholds = [1e-12, 1e-10, 1e-8, 1e-6]
    
    for threshold in thresholds
        truncated_poly, stats = truncate_polynomial_adaptive(mono_adaptive, threshold)
        sparsity_pct = round(stats.sparsity_ratio * 100, digits=1)
        @printf "  %.0e: keep %d/%d (%.1f%% sparse)\n" threshold stats.n_kept stats.n_total sparsity_pct
    end
    
    # Optimal threshold analysis
    if length(analysis.suggested_thresholds) > 0
        optimal_threshold = analysis.suggested_thresholds[1]
        truncated_optimal, stats_optimal = truncate_polynomial_adaptive(mono_adaptive, optimal_threshold)
        
        @printf "\n🎯 Optimal (%.0e): %d→%d terms (%.1f%% reduction)\n" optimal_threshold stats_optimal.n_total stats_optimal.n_kept (1-stats_optimal.sparsity_ratio)*100
    end
else
    println("\n⚠️ Skipping coefficient analysis (no valid coefficients)")
end

### 🔬 Performance Profiling

In [ ]:
# Performance profiling (optional - only if AdaptivePrecision works)
if pol_adaptive !== nothing
    println("\n🔬 Performance Profiling")
    
    using Profile
    Profile.clear()
    
    # Profile construction
    println("Profiling AdaptivePrecision construction...")
    @profile for i in 1:3
        pol = Constructor(TR, test_degree, basis=:chebyshev, precision=AdaptivePrecision, verbose=0)
    end
    
    # Show top functions
    println("\n📈 Top functions by time:")
    Profile.print(maxdepth=8, mincount=5)
    
    println("\n💡 Use ProfileView.view() for interactive analysis")
else
    println("\n⚠️ Skipping profiling (AdaptivePrecision failed)")
end

## 🎯 Phase 2B: Systematic Parameter Study

In [ ]:
# Systematic parameter study
println("\n🎯 Systematic Parameter Study")

study_results = []

for degree in degrees_to_test
    for samples in samples_to_test
        @printf "Testing: degree=%d, samples=%d..." degree samples
        
        try
            # Create test input
            TR_study = test_input(
                shubert_4d,
                dim = n,
                center = center,
                GN = samples,
                sample_range = scale_factor,
                degree_max = degree + 4
            )
            
            # Quick benchmark (fewer samples for speed)
            bench_f64 = @benchmark Constructor($TR_study, $degree, basis=:chebyshev, precision=Float64Precision, verbose=0) samples=2
            bench_adaptive = @benchmark Constructor($TR_study, $degree, basis=:chebyshev, precision=AdaptivePrecision, verbose=0) samples=2
            
            # Get actual polynomials for comparison
            pol_f64 = Constructor(TR_study, degree, basis=:chebyshev, precision=Float64Precision, verbose=0)
            pol_adaptive = Constructor(TR_study, degree, basis=:chebyshev, precision=AdaptivePrecision, verbose=0)
            
            # Store results
            result = (
                degree = degree,
                samples = samples,
                time_overhead = median(bench_adaptive.times) / median(bench_f64.times),
                memory_overhead = bench_adaptive.memory / bench_f64.memory,
                norm_difference = abs(pol_adaptive.nrm - pol_f64.nrm),
                coeffs_count = length(pol_adaptive.coeffs)
            )
            
            push!(study_results, result)
            @printf " ✅ %.2fx overhead\n" result.time_overhead
            
        catch e
            println(" ❌ Failed: $e")
        end
    end
end

@printf "\n🎉 Study completed: %d successful test cases\n" length(study_results)

### 📊 Results Analysis and Visualization

In [ ]:
# Convert results to DataFrame for analysis
if !isempty(study_results)
    results_df = DataFrame(study_results)
    
    println("📊 Parameter Study Results Summary:")
    println("=" ^ 40)
    
    # Overall statistics
    println("\n📈 Performance Statistics:")
    println("  Mean time overhead: $(round(mean(results_df.time_overhead), digits=3))x")
    println("  Median time overhead: $(round(median(results_df.time_overhead), digits=3))x")
    println("  Min time overhead: $(round(minimum(results_df.time_overhead), digits=3))x")
    println("  Max time overhead: $(round(maximum(results_df.time_overhead), digits=3))x")
    
    println("\n💾 Memory Statistics:")
    println("  Mean memory overhead: $(round(mean(results_df.memory_overhead), digits=3))x")
    println("  Median memory overhead: $(round(median(results_df.memory_overhead), digits=3))x")
    
    println("\n🎯 Accuracy Statistics:")
    println("  Mean L2 norm difference: $(mean(results_df.norm_difference))")
    println("  Max L2 norm difference: $(maximum(results_df.norm_difference))")
    println("  Cases with identical norms: $(sum(results_df.norm_difference .< 1e-15))")
    
    # Detailed breakdown by degree
    println("\n📋 Breakdown by Degree:")
    for degree in unique(results_df.degree)
        degree_data = filter(row -> row.degree == degree, results_df)
        avg_overhead = mean(degree_data.time_overhead)
        println("  Degree $degree: $(nrow(degree_data)) tests, avg overhead $(round(avg_overhead, digits=3))x")
    end
    
    # Detailed breakdown by sample size
    println("\n📊 Breakdown by Sample Size:")
    for samples in unique(results_df.samples)
        sample_data = filter(row -> row.samples == samples, results_df)
        avg_overhead = mean(sample_data.time_overhead)
        println("  Samples $samples: $(nrow(sample_data)) tests, avg overhead $(round(avg_overhead, digits=3))x")
    end
    
    # Display full results table
    println("\n📋 Complete Results Table:")
    show(results_df, allrows=true)
    
else
    println("⚠️  No results to analyze")
end

## 🔄 Phase 2C: Interactive Development Section

**This section is for iterative development and testing**

Use this space to:
- Test modifications to AdaptivePrecision implementation
- Experiment with different parameters
- Profile specific bottlenecks
- Validate accuracy improvements

**Revise.jl will automatically reload Globtim changes!**

In [ ]:
# Interactive development cell - modify as needed
println("🔄 Interactive Development Section")
println("=" ^ 40)

# Quick test after making changes to Globtim
# Revise will automatically reload changes

# Example: Test a specific configuration
test_config = (
    degree = 8,
    samples = 150,
    function_name = :shubert
)

println("🧪 Testing configuration: $test_config")

# Quick comparison
try
    result, pol_f64, pol_adaptive = compare_4d_precisions(
        test_config.function_name, 
        test_config.degree, 
        test_config.samples
    )
    
    println("✅ Test successful!")
    println("  Time overhead: $(round(result[:precision_overhead], digits=3))x")
    println("  L2 norms: F64=$(round(result[:float64_norm], digits=6)), Adaptive=$(round(result[:adaptive_norm], digits=6))")
    println("  Coefficients: $(result[:adaptive_coeffs])")
    
catch e
    println("❌ Test failed: $e")
end

println("\n💡 Modify Globtim code and re-run this cell to test changes!")

## 📋 Development Notes and Next Steps

**Use this section to track:**
- Performance bottlenecks identified
- Accuracy improvements observed
- Parameter combinations that work best
- Ideas for further optimization

### 🎯 Key Findings:
*(Update as you develop)*

### 🚀 Next Steps:
*(Update based on results)*

### 🔧 Optimization Ideas:
*(Track ideas for improvement)*